In [21]:
import pandas as pd
import numpy as np
import glob

In [30]:
file_dfs = []
for onefile in glob.glob("/Users/ahmedabukar/Downloads/:/data/*,*.csv"):
    filename = onefile.removeprefix("/Users/ahmedabukar/Downloads/:/data/")
    print(f"working on {filename}")
    
    city, state = (filename.removesuffix(".csv").split(","))
    city = city.replace("+", " ")


    df = (pd.read_csv(onefile, 
                     usecols=[1, 2, 19],                    
                      header=0, 
                      names=["max_temp", "min_temp", "precipMM"])
          .assign(city=city.title(), state=state.upper()))
    file_dfs.append(df)

working on san+francisco,ca.csv
working on new+york,ny.csv
working on springfield,ma.csv
working on boston,ma.csv
working on springfield,il.csv
working on albany,ny.csv
working on los+angeles,ca.csv
working on chicago,il.csv


In [31]:
df = pd.concat(file_dfs)
df

,max_temp,min_temp,precipMM,city,state
0,13,8,0.0,San Francisco,CA
1,13,8,0.0,San Francisco,CA
2,13,8,0.0,San Francisco,CA
3,13,8,0.0,San Francisco,CA
4,13,8,0.0,San Francisco,CA
...,...,...,...,...,...
723,3,-2,0.0,Chicago,IL
724,3,-2,0.0,Chicago,IL
725,3,-2,0.0,Chicago,IL
726,3,-2,0.0,Chicago,IL


In [48]:
def precipitation_atleast_15(df, min_mm, times):
    return df.loc[df['precipMM'] >= min_mm, "precipMM"].count() >= times

In [63]:
def has_low_temp(df):
    return (df.loc[(
        df["precipMM"] > 15) & (df["min_temp"] <= 0),
            "precipMM"]
            .count() > 3) 

In [64]:
#find cities that had at least three measurements of 10mm of precipitation or more 
#when the temperatures was at at or below 0 celsius
df.groupby(["state", "city"]).filter(has_low_temp)

,max_temp,min_temp,precipMM,city,state


In [74]:
# for each precipitation measurement, calculate the proportion of that city's total
# precipitation
df["precip_pct"] = (df.groupby(["state", "city"])
 ["precipMM"]
 .transform(lambda x : x / x.sum()))

In [83]:
# for each city, determine the greatest proportion of that city's total precipitaion
# to fall in a given period
df.groupby(["state", "city"])["precip_pct"].transform(lambda x : x.max())

0      0.056509
1      0.056509
2      0.056509
3      0.056509
4      0.056509
         ...   
723    0.057257
724    0.057257
725    0.057257
726    0.057257
727    0.057257
Name: precip_pct, Length: 5824, dtype: float64

In [84]:
df.groupby(["state", "city"])["precip_pct"].max()

state  city         
CA     Los Angeles      0.059242
       San Francisco    0.056509
IL     Chicago          0.057257
       Springfield      0.050540
MA     Boston           0.048302
       Springfield      0.060606
NY     Albany           0.029228
       New York         0.055149
Name: precip_pct, dtype: float64

In [86]:
#implement the first version of precipitation_atleast_15 using lambda?
df.groupby(["state", "city"]).filter(lambda df_ : (df_.loc[(
        df_["precipMM"] > 15) & (df_["min_temp"] <= 0),
            "precipMM"]
            .count() > 3))

,max_temp,min_temp,precipMM,city,state,precip_pct


In [88]:
#implement the second version of precipitation_atleast_15 using lambda?
df.groupby(["state", "city"]).filter(lambda df_, precip_mm, min_temp, times: (df_.loc[(
        df_["precipMM"] > precip_mm) & (df_["min_temp"] <= min_temp),
            "precipMM"]
            .count() > times), precip_mm=15, min_temp=10, times=3)

,max_temp,min_temp,precipMM,city,state,precip_pct
